In [3]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests

- Create array of strings \n seperated
- code need to be updated (not optimised one sequential things neeed to implement nparray with broadcast)

In [4]:
a_file = "/content/drive/MyDrive/Classification/CodeMixcorpora/test.txt"
response = open(a_file).read()
response = response.lower()


In [45]:
list_of_lists = []

for line in response:
  #stripped_line = line.strip()
  #line_list = stripped_line.split()

  list_of_lists.append(line)

In [46]:
len(list_of_lists)

8835222

In [5]:
response[:1500]


'" <unk> bharat me ajaad he , lekin china se tibbat tabhi ajaad hoga jab <unk> fight karenge , jo mujhe ni lagta , kyunki wo bharat ko hi apna desh manane lage he , active movement ni he unka "\nindian army kashmiri logo ko nahi atankvadiyo ko mar rahi hai\nabbe keyo fek raha hai andhe news dalal\nmather chooo dukan laga ke baitha rahta hai . digital indiaa . enn sab ko ban . karna cha he ye\njab modiji ne mudda utha hi diya ha to isko international community ke <unk> bhi jordar <unk> se uthaye ki pak baluch ke sath kaisa\nab kasmir pe rone wale apni army ko kuchh kahenge hum yha <unk> gun use ker rhe to badi jal rhi waha to real gun use ho rhi\njaldi sena vejo ............ ab wo time ageya ... now baluchistan want freedom\nvote to maine es bar bhi modi jee ko hi diya tha agee bhi bjp ko hi jayega but kuch sahi to ho jispe bitta hai wahi samjega na ankush bahi ko dekhiye gali deker ser samaj rahe hai\nmr modi you are on right path\nab pakistan ka namo nishan mitne ka din nazdeek aa gya

In [49]:
print(type(response))

<class 'str'>


In [6]:
data = response.split('\n')
data[0]

'" <unk> bharat me ajaad he , lekin china se tibbat tabhi ajaad hoga jab <unk> fight karenge , jo mujhe ni lagta , kyunki wo bharat ko hi apna desh manane lage he , active movement ni he unka "'

In [7]:
data = data[253:]
data[0]

"mithai kiyo di pakistani ranger 's ko thora sa chitta de dete punjab sarkar se lekar"

['/', 'c', 'o', 'n', 't', 'e', 'n', 't', '/', 'd', 'r', 'i', 'v', 'e', '/', 'M', 'y', 'D', 'r', 'i', 'v', 'e', '/', 'C', 'l', 'a', 's', 's', 'i', 'f', 'i', 'c', 'a', 't', 'i', 'o', 'n', '/', 'C', 'o', 'd', 'e', 'M', 'i', 'x', 'c', 'o', 'r', 'p', 'o', 'r', 'a', '/', 't', 'e', 's', 't', '.', 't', 'x', 't']


In [8]:
data = " ".join(data)
data[:2]

'mi'

- Simple cleaning and tokenisation function

In [9]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['mithai', 'kiyo', 'di', 'pakistani', 'ranger', 's', 'ko', 'thora', 'sa', 'chitta', 'de', 'dete', 'punjab', 'sarkar', 'se', 'lekar', 'him', 'log', 'pyar', 'chahte', 'he', 'leken', 'pakestan', 'unk', 'chah', 'the', 'hai', 'hmare', 'desks', 'unk', 'mare', 'jate', 'he', 'ar', 'him', 'kochh', 'nhi', 'kr', 'pate', 'hai', 'ete', 'jar', 'krte', 'hai', 'ek', 'tarah', 'indian', 'army', 'ki', 'taraf']


In [54]:
len(tokens)

1638495

In [10]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

199951


In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [13]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]


array([ 3987,  3986,   161,   615, 20912,    85,     5,  3009,   532,
       20911,   100,   368,   195,   225,    12,   735,   363,    64,
         927,  1035,    27, 20910, 20909,     1,  3985,    16,     2,
        1447, 20908,     1,   782,   542,    27,  1034,   363, 20906,
          37,    81,  3984,     2, 20905,  4822,   361,     2,    48,
         419,   121,   381,     4,   903])

In [14]:
vocab_size = len(tokenizer.word_index) + 1


In [ ]:
y = to_categorical(y, num_classes=vocab_size)


In [16]:
seq_length = X.shape[1]
seq_length

50

In [17]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [18]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [19]:
model.fit(X, y, batch_size = 256, epochs = 100)


Epoch 1/100
189/782 [======>.......................] - ETA: 5:26 - loss: 7.8088 - accuracy: 0.0240

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-924e70a18025>", line 1, in <module>
    model.fit(X, y, batch_size = 256, epochs = 100)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 855, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py", line 2943, in __call__
    filtered_flat_args, captured_inputs=graph_function.captured_inputs)  # pylint: disable=pro

KeyboardInterrupt: ignored

- Peek any line from text to work on for text generation

In [1]:
seed_text=lines[12343]
seed_text

NameError: ignored

Generate Text sequence length according to requirement

In [2]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)
